Inserting the pCTR into train and test set

In [ ]:
#train set
bid_requests = pd.read_csv('ipinyou/1458/train.log.txt', sep="\t")
ctr = pd.read_csv('ipinyou/1458/ctr_pred_train.txt', sep="\t")
bid_requests.insert(1, "pCTR", ctr.iloc[:,1])
bid_requests.to_csv('ipinyou/1458/train.log.txt', sep="\t", header=True, index=False)

In [ ]:
#test set
bid_requests = pd.read_csv('ipinyou/1458/test.log.txt', sep="\t")
ctr = pd.read_csv('ipinyou/1458/ctr_pred_test.txt', sep="\t")
bid_requests.insert(1, "pCTR", ctr.iloc[:,1])
bid_requests.to_csv('ipinyou/1458/test.log.txt', sep="\t", header=True, index=False)

Training the Logistic Regression for CTR prediction only on the "Usertag" feature (for the LinBid)

In [1]:
import pandas as pd
# Load datasets
train_usertags = pd.read_csv('ipinyou/1458/train.log.txt', sep="\t", usecols = ["usertag", "click"]) 
test_usertags = pd.read_csv('ipinyou/1458/test.log.txt', sep="\t", usecols = ["usertag", "click"]) 

In [2]:
train_tag_dummies = pd.get_dummies(train_usertags.usertag.fillna("0,").str.split(",").apply(pd.Series).stack()).sum(level=0).iloc[:,1:]
test_tag_dummies = pd.get_dummies(test_usertags.usertag.fillna("0,").str.split(",").apply(pd.Series).stack()).sum(level=0).iloc[:,1:]

In [16]:
import numpy as np
intersect_cols = np.intersect1d(train_tag_dummies.columns.values, test_tag_dummies.columns.values)

In [24]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(class_weight='balanced', max_iter=1000)

In [25]:
logreg.fit(train_tag_dummies[intersect_cols], train_usertags.click)

LogisticRegression(class_weight='balanced', max_iter=1000)

In [28]:
ctr_segment_pred = logreg.predict_proba(test_tag_dummies[intersect_cols])
ctr_set_pred_train = logreg.predict_proba(train_tag_dummies[intersect_cols])

In [42]:
pd.DataFrame(ctr_segment_pred).to_csv('test_ctr_seg_pred.tsv', sep = '\t')
ctr_seg_test = pd.read_csv('test_ctr_seg_pred.tsv', sep = '\t', usecols = ['1'])
ctr_seg_test.to_csv('ctr_seg_test.tsv', sep = '\t', index = False)

In [63]:
from sklearn.metrics import roc_auc_score
print("pCTR AUC in the test set: ", roc_auc_score(test_usertags['click'], test['1']))

pCTR AUC in the test set:  0.9787214015774867
